In [37]:
import sys
import numpy as np
import os
from PIL import Image, ImageOps
from pathlib import Path
from potrace import Bitmap, POTRACE_TURNPOLICY_MINORITY  # `potracer` library


def file_to_svg(filename: str):
    """
    Converts a raster image (PNG, JPEG, etc.) into an SVG vector graphic using the Potrace algorithm.

    This function loads a given image file, applies histogram equalization to enhance contrast,
    then traces the resulting bitmap to generate smooth vector paths. The output is saved as an
    SVG file with the same base filename as the input.

    Parameters:
    -----------
    filename : str
        Path to the input image file.

    Notes:
    ------
    - Uses the `potracer` library's Bitmap tracing functionality.
    - Outputs a single black-filled SVG path representing the image silhouette or edge detail.
    - Histogram equalization via PIL's ImageOps helps normalize brightness levels.
    - The `blacklevel` and `alphamax` parameters affect the tracing fidelity.

    Output:
    -------
    Saves an SVG file to the same directory as the input image.

    Example:
    --------
    >>> file_to_svg("curtain_photo.png")
    # Creates "curtain_photo.svg" in the same directory.
    """

    try:
        print(f'Loading image {filename}...')

        image = Image.open(filename)
        image = ImageOps.equalize(image)
        # image = image.convert("L")  # Convert to 8-bit grayscale
        # image = image.convert("1")  # 1-bit black and white (hard threshold)


        # avg = int(np.array(image).mean())
        # image = image.point(lambda p: 0 if p < avg else 255, "1")

    except Exception as e:
        print(f'Failed to load image {filename}: {e}')
        return
    bm = Bitmap(image, blacklevel=.250)
    # bm.invert()
    plist = bm.trace(
        turdsize=1,
        turnpolicy=POTRACE_TURNPOLICY_MINORITY,
        alphamax=20,
        opticurve=False,
        opttolerance=0.4,
    )
    with open(f"{os.path.splitext(filename)[0]}.svg", "w") as fp:
        fp.write(
            f'''<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{image.width}" height="{image.height}" viewBox="0 0 {image.width} {image.height}">''')
        parts = []
        for curve in plist:
            fs = curve.start_point
            parts.append(f"M{fs.x},{fs.y}")
            for segment in curve.segments:
                if segment.is_corner:
                    a = segment.c
                    b = segment.end_point
                    parts.append(f"L{a.x},{a.y}L{b.x},{b.y}")
                else:
                    a = segment.c1
                    b = segment.c2
                    c = segment.end_point
                    parts.append(f"C{a.x},{a.y} {b.x},{b.y} {c.x},{c.y}")
            parts.append("z")
        fp.write(f'<path stroke="none" fill="black" fill-rule="evenodd" d="{"".join(parts)}"/>')
        fp.write("</svg>")

#
# if __name__ == '__main__':
#     file_to_svg(sys.argv[1])


source_dir = Path('/mnt/mls/data/gabemcwilliams_info')
file_to_svg(source_dir / 'stage_main.png')

Loading image /mnt/mls/data/gabemcwilliams_info/stage_main.png...
